In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook, save
from bokeh.plotting import figure
from bokeh.charts import Line, Bar
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
output_notebook()

Loading BokehJS ...

In [3]:
# Total loan amount you will pay
loan_amount = 250000
# Percentage you will put down(as a decimal 5% == 0.05)
down_payment_pct = .05
# Expected number of months you will live in house
expected_length = 4*12
# Loan Length(in years)
loan_length = 15
# The rates you want to look at(Base comparison rate needs to be first)
rates = [0.0250, 0.0275, 0.02625, 0.02875, 0.03, 0.03125]
#rates = [0.0250, 0.03]
# Points costs for rates(must be same size as rates. negative numbers mean you are paid that amount for the loan)
loan_costs = [2375, 0, 1187.5, -1781.25, -2968.75, -4156.25]
#loan_costs = [2375, -2968.75]
assert len(rates) == len(loan_costs), 'Make sure rates and loan_costs have same length'
# Rates and costs together
rates_and_costs = zip(rates, loan_costs)
# The principal of the loan to be paid
principal_amount = loan_amount * (1-down_payment_pct)

print "Principal Amount: {0}".format(principal_amount)

Principal Amount: 237500.0


# Interest and Payment portions of payment

In [4]:
def compare(rate1, cost1, rate2, cost2):
    rates_and_costs = [(rate1, cost1), (rate2, cost2)]
    # Financial start the period count at 1 so that is why + 1
    per = np.arange(12*loan_length) + 1
    cumulative_interest_paid = cip = {}
    monthly_payments = mp = []
    for rate, cost in rates_and_costs:
        # Interest payments over course of loan(every month)
        ipmts = -1 * np.ipmt(rate/12, per, 12*loan_length, principal_amount)
        # Principal payments over course of loan(every month)
        ppmts = -1 * np.ppmt(rate/12, per, 12*loan_length, principal_amount)
        # Calculates monthly payments of loan(every month)
        pmts = ipmts + ppmts
        # Cumulitive interest paid for each month
        cip[str(rate)] = np.cumsum(ipmts) + cost
        lines = {
            'Interest Pmt': ipmts,
            'Principal': ppmts,
            'Payment': pmts
        }
        # Holds monthly payments
        mp.append(pmts[0])
        #p = Line(lines, title='Rate: {0}'.format(rate))
        #show(p)
    cost_diff = cost1 - cost2
    mp_diff = np.absolute(mp[0] - mp[1])
    break_even_mp = cost_diff / mp_diff
    print "{0} - {1} = {2}".format(cost1, cost2, cost_diff)
    print "{0} - {1} = {2}".format(mp[0], mp[1], mp_diff)
    print "{0} / {1} = {2}".format(cost_diff, mp_diff, break_even_mp)
    title='{0} vs {1}'.format(rate1, rate2)
    p = figure(title=title)
    colors = ['red', 'green', 'blue', 'orange']
    color_map = dict(zip(cip.keys(), colors))
    for k,v in cip.iteritems():
        p.line(x=range(len(v)), y=v, legend=str(k), line_color=color_map[k])
    vals = np.absolute(np.subtract(*cip.values()))
    break_even_months = vals.argmin()
    break_even_value = cip.values()[0][break_even_months]
    break_even_point = Label(x=break_even_months, y=break_even_value, text='{0}m: {1}y'.format(break_even_months, break_even_months/12.0))
    break_even_mp_point = Label(x=break_even_mp, y=2, text='{0}m: {1}y'.format(break_even_mp, break_even_mp/12.0))
    p.add_layout(break_even_point)
    p.add_layout(break_even_mp_point)
    #p.add_layout(labels)
    show(p)

for i in range(len(rates)):
    rate = rates[i]
    cost = loan_costs[i]
    print rate, cost
    for r,c in rates_and_costs[i+1:]:
        print r, c
        compare(rate, cost, r, c)

0.025 2375
0.0275 0
2375 - 0 = 2375
1583.6243714 - 1611.72638889 = 28.1020174905
2375 / 28.1020174905 = 84.5135051531


0.02625 1187.5
2375 - 1187.5 = 1187.5
1583.6243714 - 1597.63744301 = 14.0130716143
1187.5 / 14.0130716143 = 84.7423058046


0.02875 -1781.25
2375 - -1781.25 = 4156.25
1583.6243714 - 1625.89108334 = 42.2667119425
4156.25 / 42.2667119425 = 98.3338852016


0.03 -2968.75
2375 - -2968.75 = 5343.75
1583.6243714 - 1640.13139566 = 56.507024264
5343.75 / 56.507024264 = 94.5678890298


0.03125 -4156.25
2375 - -4156.25 = 6531.25
1583.6243714 - 1654.44719016 = 70.8228187614
6531.25 / 70.8228187614 = 92.2195715198


...

0.03125 -4156.25
1187.5 - -4156.25 = 5343.75
1597.63744301 - 1654.44719016 = 56.8097471471
5343.75 / 56.8097471471 = 94.0639638153


0.02875 -1781.25
0.03 -2968.75
-1781.25 - -2968.75 = 1187.5
1625.89108334 - 1640.13139566 = 14.2403123215
1187.5 / 14.2403123215 = 83.390024965


0.03125 -4156.25
-1781.25 - -4156.25 = 2375.0
1625.89108334 - 1654.44719016 = 28.5561068189
2375.0 / 28.5561068189 = 83.1696006414


0.03 -2968.75
0.03125 -4156.25
-2968.75 - -4156.25 = 1187.5
1640.13139566 - 1654.44719016 = 14.3157944974
1187.5 / 14.3157944974 = 82.9503385381


0.03125 -4156.25


In [5]:
monthly_payments = np.pmt(np.array(rates)/12, 12*loan_length, principal_amount)
print "Monthly Payments: {0}".format(-1*monthly_payments)

Monthly Payments: [ 1583.6243714   1611.72638889  1597.63744301  1625.89108334  1640.13139566
  1654.44719016]


In [6]:
monthly_totals = [
    np.arange(loan_cost, -1*monthly_payment * (loan_length*12), -1*monthly_payment) 
    for rate, loan_cost, monthly_payment in zip(rates, loan_costs, monthly_payments)
]
lengths = [len(x) for x in monthly_totals]
max_length = max(lengths)
for i in range(len(monthly_totals)):
    x = monthly_totals[i]
    if len(x) != max_length:
        t = [x[-1]] * (max_length - len(x))
        monthly_totals[i] = np.append(monthly_totals[i], t)
    
#print monthly_totals
monthly_totals = dict(zip([str(x) for x in rates], monthly_totals))

In [7]:
# The graphic below shows you over time where each of the rates vs points will get you
# The lower the line the better
l = Line(monthly_totals, title="Rates vs Points", ylabel='Amount Paid', xlabel='Month')
show(l)

In [8]:
x_y = zip([np.linspace(0, 15, 15*12) for i in range(len(monthly_totals))], monthly_totals)

In [9]:
p = figure(title="simple line example", plot_height=300, plot_width=600)
r = p.line(monthly_totals, line_width=3)

ValueError: expected an element of either String, Dict(String, Either(String, Instance(Transform), Instance(ColorMapper), Float)) or Float, got {'0.03125': array([  -4156.25      ,   -2501.80280984,    -847.35561968,
           807.09157047,    2461.53876063,    4115.98595079,
          5770.43314095,    7424.8803311 ,    9079.32752126,
         10733.77471142,   12388.22190158,   14042.66909173,
         15697.11628189,   17351.56347205,   19006.01066221,
         20660.45785236,   22314.90504252,   23969.35223268,
         25623.79942284,   27278.246613  ,   28932.69380315,
         30587.14099331,   32241.58818347,   33896.03537363,
         35550.48256378,   37204.92975394,   38859.3769441 ,
         40513.82413426,   42168.27132441,   43822.71851457,
         45477.16570473,   47131.61289489,   48786.06008504,
         50440.5072752 ,   52094.95446536,   53749.40165552,
         55403.84884568,   57058.29603583,   58712.74322599,
         60367.19041615,   62021.63760631,   63676.

In [ ]:
'''
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()
'''

In [ ]:
show(p, notebook_handle=True)

In [ ]:
#interact(update, f=["sin", "cos", "tan"], w=(0,100), A=(1,5), phi=(0, 20, 0.1))